# Imaging observations of HL Tau

The purpose of this notebook is to show how to use a FITS image as input to create a `Source` object for imaging observations with METIS. We work with an ALMA 233 GHz image of HL Tau, pretending that this is the structure of the object in the mid-infrared. It will be joined to a very simple spectrum. The image is already prepared such that pixel values represent Jy/pixel in the observed band.

In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy import units as u
from astropy.wcs import WCS
from synphot import SourceSpectrum
from synphot.models import Empirical1D, ConstFlux1D

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
import scopesim as sim

In [ ]:
sim.bug_report()

To simulate observations with METIS, the instrument packages for METIS, the ELT, and Armazones are required. The packages are downloaded from the server and installed into sub-directory `inst_pkgs` in the current working directory.

In [ ]:
sim.download_package(['instruments/METIS', 'telescopes/ELT', 'locations/Armazones'])

## Creating a source object from an image and a spectrum

The image that we use is `HL_Tau_prep_for_Scopesim.fits`. This has been derived from `HLTau_B6cont_mscale_ap.image.fits`, which was retrieved from the ESO Science archive. The main step in preparing this file was a rescaling of the image values such that they directly represent flux in Jy per pixel (the flux scale was arbitrarily set such that the average over the first ring corresponds to 0.01 Jy/arcsec2). The image has a pixel scale of 5 mas, which was retained.  

In [ ]:
hdul = fits.open("HL_Tau_prep_for_Scopesim.fits")

The header contains the information necessary for Scopesim, a WCS and the `BUNIT` keyword that gives the units of the pixel values. Scopesim mainly needs the `CDELT` keywords (or the `CD` matrix if present), which contain the pixel scale of the input image. The reference coordinates `CRVAL` have been set to zero as other values may confuse Scopesim at this stage.

In [ ]:
hdul[1].header

A `Source` object in ScopeSim currently consists of an image (given as a FITS `HDU`) and a spectrum. The interface will be simplified to create the spectrum internally and take flux values from the image. Hence the next couple of cells will be superfluous in the future.

In [ ]:
wave = np.linspace(2.5, 18.5, 1001) * u.um
flux = np.ones_like(wave.value) * 1 * u.Jy

spec = SourceSpectrum(Empirical1D, points=wave, lookup_table=flux)

Now we can finally create our `Source` object:

In [ ]:
src = sim.Source(spectra=[spec], image_hdu=hdul[1])

## N-band observation of the source

We set up the instrument to the N-band imaging mode.

In [ ]:
cmd_n = sim.UserCommands(use_instrument='METIS', set_modes=['img_n'])

The default filter is N2. It could be changed by modifying the configuration keyword

In [ ]:
cmd_n['!OBS.filter_name']

The instrument itself (the `OpticalTrain`), including atmosphere and telescope, is built by

In [ ]:
metis_n = sim.OpticalTrain(cmd_n)
metis_n["chopnod"].include

The optical train contains the following effects:

In [ ]:
metis_n.effects

We can now "observe" the source. This command creates the ideal image just in front of the detector.

In [ ]:
metis_n.observe(src, update=True)

The N band configuration of METIS includes a detector effect `ChopNodCombiner`. This creates four detector images with chopping and nodding offsets and combines them into a chop-nod image with positive and negative beam images. The default is parallel chopping and nodding with

In [ ]:
print(metis_n.cmds['!OBS.chop_offsets'], metis_n.cmds['!OBS.nod_offsets'])

ScopeSim can automatically determine DIT and NDIT based on your choice of exptime and filter.

In [ ]:
outhdul_parallel = metis_n.readout(exptime=3600, detector_readout_mode="auto")[0]

Before writing the result to disk, we rescale to one DIT and add a little bit of information to the header.

In [ ]:
outhdul_parallel[1].data /= 4 * ndit
outhdul_parallel[0].header['DIT'] = dit
outhdul_parallel[0].header['NDIT'] = ndit
outhdul_parallel[0].header['EXPTIME'] = dit
outhdul_parallel[0].header['INTTIME'] = 4 * dit * ndit
outhdul_parallel.writeto("hl_tau_metis_n_img_parallel.fits", overwrite=True)

In [ ]:
fig = plt.figure(figsize=(10, 2))
plt.imshow(outhdul_parallel[1].data[750:1300,:], origin='lower')#, norm=LogNorm(), vmin=3e11, vmax=5e11)
plt.colorbar()

To change to perpendicular chopping and nodding, `nod_offsets` needs to operate in the y-direction:

In [ ]:
metis_n.cmds['!OBS.nod_offsets'] = [0, 3]

In [ ]:
outhdul_perpendicular = metis_n.readout()[0]

In [ ]:
outhdul_perpendicular[1].data /= 4 * ndit
outhdul_perpendicular[0].header['DIT'] = dit
outhdul_perpendicular[0].header['NDIT'] = ndit
outhdul_perpendicular[0].header['EXPTIME'] = dit
outhdul_perpendicular[0].header['INTTIME'] = 4 * dit * ndit
outhdul_perpendicular.writeto('hl_tau_metis_n_img_perpendicular.fits', overwrite=True)

In [ ]:
fig = plt.figure(figsize=(10, 8))
plt.imshow(outhdul_perpendicular[1].data[750:1750, 750:1750], origin='lower')
plt.colorbar()

## Dithered L-band observations of the source

The spectrum of our source object was defined on a wavelength range that makes the source immediately suitable for L-band observations. We have to set up the instrument from scratch:

In [ ]:
cmd_l = sim.UserCommands(use_instrument="METIS", set_modes=["img_lm"])

The default filter is L':

In [ ]:
cmd_l["!OBS.filter_name"]

In [ ]:
metis_l = sim.OpticalTrain(cmd_l)

In [ ]:
metis_l['detector_linearity'].include = False

In [ ]:
metis_l.observe(src, update=True)

For the investigation of science cases, it should mostly be sufficient to set DIT and NDIT to represent the entire observation in one go.

In [ ]:
metis_l.cmds['!OBS.dit'] = 0.040   # MINDIT of H2RG: 40 ms
metis_l.cmds['!OBS.ndit'] = 3600

In [ ]:
hdu_tot = metis_l.readout()[0]

In [ ]:
plt.imshow(hdu_tot[1].data[750:1300, 750:1300], origin='lower', norm=LogNorm())

It is possible to simulate dithered observations using the `.shift()` method of the source object. Note that this alters the source object, so any shift is relative to the current position.

In [ ]:
metis_l.cmds['!OBS.dit'] = 0.040
metis_l.cmds['!OBS.ndit'] = 600

In [ ]:
hdu_1 = metis_l.readout(src)[0]

In [ ]:
src.shift(dx=1., dy=0.)
metis_l.observe(src, update=True)
hdu_2 = metis_l.readout(src)[0]

In [ ]:
src.shift(dx=0., dy=1.)
metis_l.observe(src, update=True)
hdu_3 = metis_l.readout(src)[0]

In [ ]:
vmin, vmax = np.median(hdu_1[1].data), np.max(hdu_1[1].data)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(hdu_1[1].data[750:1500, 750:1500], origin='lower', vmin=vmin, vmax=vmax, norm=LogNorm())
axes[1].imshow(hdu_2[1].data[750:1500, 750:1500], origin='lower', vmin=vmin, vmax=vmax, norm=LogNorm())
axes[2].imshow(hdu_3[1].data[750:1500, 750:1500], origin='lower', vmin=vmin, vmax=vmax, norm=LogNorm())